In [1]:
!pip install keras==2.3.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.8/377.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.8.0
    Uninstalling keras-3.8.0:
      Successfully uninstalled keras-3.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikeras 0.13.0 requires keras>=3.2.0, but you have keras 2.3.1 which is incompatible.
tensorflow 2.18.0 requires keras>=3.5.0, but you have keras 2.3.1 which is incompatible.


In [2]:
!pip uninstall keras tensorflow -y


Found existing installation: Keras 2.3.1
Uninstalling Keras-2.3.1:
  Successfully uninstalled Keras-2.3.1
Found existing installation: tensorflow 2.18.0
Uninstalling tensorflow-2.18.0:
  Successfully uninstalled tensorflow-2.18.0


In [3]:
!pip install tensorflow --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 862.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.18.0 requires tensorflow<2.19,>=2.18, but you have tensorflow 2.19.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have

In [4]:
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
import numpy as np

In [5]:
import tensorflow as tf
print(tf.__version__)  # This should print a valid version like 2.x.x


2.19.0


In [6]:
!pip install scikeras[tensorflow]


In [7]:
from pickle import load, dump
from tensorflow.keras.utils import plot_model
from keras.models import Model
from keras.layers import *
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from scikeras.wrappers import KerasClassifier
import random
from numpy import array

In [8]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os

base_dir = "/content/drive/MyDrive/finalattention"  # Change 'your_project_folder' to any name you want

# Define required folders
folders = [
    "Description",
    "Features",
    "model/results"
]

# Create directories
for folder in folders:
    os.makedirs(os.path.join(base_dir, folder), exist_ok=True)

print(f"Folders created under: {base_dir}")


Folders created under: /content/drive/MyDrive/finalattention


In [13]:
!wget -P /content/remote_sensing_image_captioning/Dataset "https://github.com/chan64/remote_sensing_image_captioning/tree/master/Dataset"


--2025-03-12 14:51:44--  https://github.com/chan64/remote_sensing_image_captioning/tree/master/Dataset
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘/content/remote_sensing_image_captioning/Dataset/Dataset’

Dataset                 [  <=>               ] 220.02K   903KB/s    in 0.2s    

2025-03-12 14:51:45 (903 KB/s) - ‘/content/remote_sensing_image_captioning/Dataset/Dataset’ saved [225300]



In [14]:
import shutil
shutil.move("/content/remote_sensing_image_captioning/Dataset", "/content/drive/MyDrive/finalattention/Dataset")


'/content/drive/MyDrive/finalattention/Dataset'

In [15]:
dataset_path = "/content/drive/MyDrive/finalattention/Dataset"


In [17]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# load a pre-defined list of photo identifiers
def load_set(filename):
    doc = load_doc(filename)
    dataset = list()
    # process line by line
    for line in doc.split('\n'):
        # skip empty lines
        if len(line) < 1:
            continue
        # get the image identifier
        identifier = line.split(' ')[0]
        dataset.append(identifier)
    return set(dataset)

# load photo features
def load_video_features(filename, dataset):
    # load all features
    all_features = load(open(filename, 'rb'))
    # filter features
    features = {k: all_features[k][:256] for k in dataset}
    return features

In [19]:
#load training dataset (6K)
filename = '/content/drive/MyDrive/finalattention/Description/trainuc.txt'
train = load_set(filename)
print('Train: %d' % len(train))
#load training dataset (6K)
filename = '/content/drive/MyDrive/finalattention/Description/testuc.txt'
test = load_set(filename)
print('Test: %d' % len(test))
'''
#load c2d features
train_vid_features = load_video_features('dataset_one_hot_train.pkl', train)
print('C2D: train=%d' % len(train_vid_features))

#load c2d features
test_vid_features = load_video_features('dataset_one_hot_test.pkl', test)
print('C2D: train=%d' % len(test_vid_features))
class_label=load(open('/home/mh/mywork/dataset/MSVD/features/class_features/msvd_video_class_label_6.pkl','rb'))

#load c2d features
train_vid_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/msvd_sem_scn_300.pkl', train)
print('C2D: train=%d' % len(train_vid_features))

#load c2d features
test_vid_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/msvd_sem_scn_300.pkl', test)
print('C2D: train=%d' % len(test_vid_features))

'''
#load c2d features
train_vid_features = load_video_features('/content/drive/MyDrive/finalattention/Features/features_uc_resnet152_updated.pkl', train)

print('Loaded: train=%d' % len(train_vid_features))

#load c2d features
test_vid_features = load_video_features('/content/drive/MyDrive/finalattention/Features/features_uc_resnet152_updated.pkl', test)
print('Loaded: test=%d' % len(test_vid_features))
class_label=load(open('/content/drive/MyDrive/finalattention/Features/dataset_one_hot_uc.pkl','rb'))


#print(train_vid_features['05_010'])

Xtrain=list()
Ytrain=list()
for i in train_vid_features:
    Xtrain.append(train_vid_features[i][0])
    #print(i)
    #x = i.split('_')[0]
    Ytrain.append(class_label[i])

print(Xtrain[0])
Xtrain=np.array(Xtrain)
print(Xtrain.shape)

print(Ytrain[0])
Ytrain=np.array(Ytrain)
print(Ytrain.shape)

Xtest = list()
Ytest = list()
for i in test_vid_features:
    Xtest.append(test_vid_features[i][0])
    Ytest.append(class_label[i])

Xtest=np.array(Xtest)
print(Xtest.shape)

Ytest=np.array(Ytest)
print(Ytest.shape)


'''
Xtest=list()
Ytest=list()
for i in test_vid_features:
    Xtest.append(test_vid_features[i][0])
    Ytest.append(class_label[i])

Xtest=np.array(Xtest)
print(Xtest.shape)
Ytest=np.array(Ytest)
print(Ytest.shape)

#load c2d features
train_c2d_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/vlad/msvd_resnet152_vlad_features_k_100.pkl', train)
print('C2D: train=%d' % len(train_c2d_features))

#load c3d features
train_c3d_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/vlad/msvd_c3d_vlad_features_k_100.pkl', train)
print('C3D: train=%d' % len(train_c3d_features))

#load semantic features
train_semantic_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/msvd_sem_scn_300.pkl', train)
print('Semantic: train=%d' % len(train_semantic_features))

Xtrain=list()
Ytrain=list()
for i in train_c2d_features:
    c2d=np.array(train_c2d_features[i])
    c3d=np.array(train_c3d_features[i])
    sem=np.array(train_semantic_features[i])
    Xtrain.append(np.concatenate((c2d,c3d,sem),axis=0))
    Ytrain.append(Ytrain_label[i])

Xtrain=np.array(Xtrain)
print(Xtrain.shape)
Ytrain=np.array(Ytrain)
print(Ytrain.shape)

Xtest=list()
Ytest=list()
for i in test_c2d_features:
    c2d=np.array(test_c2d_features[i])
    c3d=np.array(test_c3d_features[i])
    sem=np.array(test_semantic_features[i])
    Xtest.append(np.concatenate((c2d,c3d,sem),axis=0))
    Ytest.append(Ytest_label[i])

Xtest=np.array(Xtest)
print(Xtest.shape)
Ytest=np.array(Ytest)
print(Ytest.shape)
'''

Train: 1260
Test: 420
Loaded: train=1260
Loaded: test=420
[0.09365865 0.05717707 4.2325883  ... 1.8605973  0.22527066 0.        ]
(1260, 2048)
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
(1260, 21)
(420, 2048)
(420, 21)


"\nXtest=list()\nYtest=list()\nfor i in test_vid_features:\n    Xtest.append(test_vid_features[i][0])\n    Ytest.append(class_label[i])\n\nXtest=np.array(Xtest)\nprint(Xtest.shape)\nYtest=np.array(Ytest)\nprint(Ytest.shape)\n\n#load c2d features\ntrain_c2d_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/vlad/msvd_resnet152_vlad_features_k_100.pkl', train)\nprint('C2D: train=%d' % len(train_c2d_features))\n\n#load c3d features\ntrain_c3d_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/vlad/msvd_c3d_vlad_features_k_100.pkl', train)\nprint('C3D: train=%d' % len(train_c3d_features))\n\n#load semantic features\ntrain_semantic_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/msvd_sem_scn_300.pkl', train)\nprint('Semantic: train=%d' % len(train_semantic_features))\n\nXtrain=list()\nYtrain=list()\nfor i in train_c2d_features:\n    c2d=np.array(train_c2d_features[i])\n    c3d=np.array(train_c3d_features[i])\n    sem=np.array(train

In [21]:
features1 = load(open('/content/drive/MyDrive/finalattention/Features/dataset_one_hot_newids.pkl','rb'))
print((features1.keys()))

dict_keys(['100_050', '106_105', '103_068', '111_144', '107_028', '109_002', '104_042', '106_138', '106_118', '104_002', '106_059', '100_082', '108_073', '112_088', '100_051', '109_127', '111_054', '112_071', '112_097', '107_013', '102_044', '104_024', '103_019', '110_039', '106_110', '100_038', '100_124', '108_131', '112_010', '100_071', '109_150', '110_051', '104_084', '107_026', '110_052', '106_104', '101_013', '108_096', '108_150', '106_143', '103_059', '109_078', '103_018', '100_006', '100_091', '105_027', '106_120', '103_039', '106_094', '108_032', '105_007', '102_066', '103_143', '108_137', '109_114', '108_021', '100_149', '110_042', '111_011', '103_074', '102_069', '103_058', '100_005', '108_109', '104_082', '104_073', '103_114', '106_003', '109_147', '104_029', '108_055', '109_124', '111_121', '104_069', '108_124', '100_137', '103_139', '100_069', '111_058', '108_136', '109_108', '106_139', '103_144', '100_022', '112_014', '105_018', '112_056', '102_028', '103_078', '101_027',

In [22]:

# data generator, intended to be used in a call to model.fit_generator()
def data_generator(X,Y,batch_size):
    # loop for ever over images
    batch_features = np.zeros((batch_size, 830))
    batch_labels = np.zeros((batch_size,14))
    while 1:
        for i in range(batch_size):
            # choose random index in features
            index= random.choice(list(X.keys()))
            batch_features[i] = X[index]
            batch_labels[i] = Y[index]
        yield batch_features, batch_labels
'''

# data generator, intended to be used in a call to model.fit_generator()
def data_generator(train_c2d, train_c3d, train_sem,y_label):
    X1, X2, X3, y = list(), list(), list(), list()
    # loop for ever over images
    while 1:
        for key, desc_list in train_c2d.items():
            # retrieve the photo feature
            c2d= train_c2d[key]
            X1.append(c2d)
            c3d= train_c3d[key]
            X2.append(c3d)
            sem=train_sem[key]
            X3.append(sem)
            y.append(y_label[key])
            #print(len(photo))
            yield [[array(X1), array(X2), array(X3)], array(y)]
'''

'\n\n# data generator, intended to be used in a call to model.fit_generator()\ndef data_generator(train_c2d, train_c3d, train_sem,y_label):\n    X1, X2, X3, y = list(), list(), list(), list()\n    # loop for ever over images\n    while 1:\n        for key, desc_list in train_c2d.items():\n            # retrieve the photo feature\n            c2d= train_c2d[key]\n            X1.append(c2d)\n            c3d= train_c3d[key]\n            X2.append(c3d)\n            sem=train_sem[key]\n            X3.append(sem)\n            y.append(y_label[key])\n            #print(len(photo))\n            yield [[array(X1), array(X2), array(X3)], array(y)]\n'

In [24]:
# define baseline model
def baseline_model():
    # feature extractor model
    inputs1 = Input(shape=(2048,))
    #fe1 = Dropout(0.2)(inputs1)
    fe1 = Dense(256, activation='relu')(inputs1)
    #print(fe1)
    outputs = Dense(21, activation='softmax')(fe1)
    # tie it together [image, seq] [word]
    model = Model(inputs=inputs1, outputs=outputs)
        # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize model
    model.summary()
    plot_model(model, to_file='/content/drive/MyDrive/finalattention/model/model_classify_domain.png', show_shapes=True)
    return model

In [25]:
model=baseline_model()
epochs = 50
batch_size=1260
steps=len(train_vid_features)
#print(i)

    # create the data generator
    #generator = data_generator(train_vid_features,Ytrain_label,batch_size)
    # fit for one epoch
    #model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    # model.evaluate_generator(generator=valid_generator)
history = model.fit(Xtrain, Ytrain,validation_split=0.20, batch_size=batch_size, epochs=150)
    # save model
model.save('/content/drive/MyDrive/finalattention/model/model.h5')

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         524,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 21)                  │           5,397 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 529,941 (2.02 MB)

 Trainable params: 529,941 (2.02 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0308 - loss: 3.7433 - val_accuracy: 0.2222 - val_loss: 2.6701
Epoch 2/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.2768 - loss: 2.4540 - val_accuracy: 0.4206 - val_loss: 2.0165
Epoch 3/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 148ms/step - accuracy: 0.5149 - loss: 1.7119 - val_accuracy: 0.5873 - val_loss: 1.4338
Epoch 4/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step - accuracy: 0.7222 - loss: 1.1445 - val_accuracy: 0.7540 - val_loss: 0.9982
Epoch 5/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.8333 - loss: 0.7789 - val_accuracy: 0.8294 - val_loss: 0.7178
Epoch 6/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.8968 - loss: 0.5541 - val_accuracy: 0.8730 - val_loss: 0.5514
Epoch 7/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 320ms/step - accuracy: 0.9216 - loss: 0.4090 - val_accuracy: 0.8849 - val_loss: 0.4530
Epoch 8/150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 311ms/step - accuracy: 0.9286 - loss: 0.3162 - val_accuracy: 0.8929 - val_

In [27]:
# Visualize training history
import matplotlib.pyplot as plt
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('/content/drive/MyDrive/finalattention/model/accuracy.png')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.savefig('/content/drive/MyDrive/finalattention/model/loss.png')
plt.show()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


KeyError: 'acc'

# Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
'''
model = load_model('/home/mh/mywork/classification/model_classify/model_4.h5')

Ytest_label = load(open('/home/mh/mywork/dataset/MSVD/features/class_features/msvd_video_class_label_test.pkl', 'rb'))
filename = '/home/mh/mywork/dataset/MSVD/new_testID.txt'
test = load_set(filename)
print('Dataset: %d' % len(test))

#load c2d features
test_c2d_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/vlad/msvd_resnet152_vlad_features_k_100.pkl', test)
print('C2D: test=%d' % len(test_c2d_features))

#load c3d features
test_c3d_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/vlad/msvd_c3d_vlad_features_k_100.pkl', test)
print('C3D: test=%d' % len(test_c3d_features))

#load semantic features
test_semantic_features = load_video_features('/home/mh/mywork/dataset/MSVD/features/msvd_sem_scn_300.pkl', test)
print('Semantic: test=%d' % len(test_semantic_features))
'''

In [ ]:

model = load_model('drive/My Drive/Final Year Project/modelcom/model_01_077.h5')
#features=load(open('features_resnet152_combined.pkl','rb'))
#print(len(features))

#accuracy for test
predictions=list()
pr_test=dict()
y_label=list()
count=0
for i in test_vid_features:
    #print(i)
    x_n=np.array([test_vid_features[i]][0])
    #score = model.evaluate(x, y, batch_size=batch_size)
    preds=model.predict([x_n])
    labels = np.argmax(preds, axis=-1)
    predictions.append(labels[0])
    y_label.append(class_label[i])
    pr_test[count]=i
    count+=1

predictions_test=np.array(predictions)
ytest=np.array(y_label)
pred_label = preprocessing.LabelBinarizer()
test_predicted_label=pred_label.fit_transform(predictions_test)

Ytest=Ytest.astype(int)

#accuracy for train:
predictions=list()
pr_train=dict()
y_label=list()
count=0
for i in train_vid_features:
    #print(i)
    x_n=np.array(train_vid_features[i])

    #score = model.evaluate(x, y, batch_size=batch_size)
    preds=model.predict([x_n])
    labels = np.argmax(preds, axis=-1)
    predictions.append(labels[0])
    y_label.append(class_label[i])
    pr_train[count]=i
    count+=1

predictions_train=np.array(predictions)
ytrain=np.array(y_label)
pred_label = preprocessing.LabelBinarizer()
train_predicted_label=pred_label.fit_transform(predictions_train)

Ytrain=Ytrain.astype(int)

In [ ]:
print((predictions_train))

In [ ]:
cm = confusion_matrix(Ytest.argmax(axis=1), test_predicted_label.argmax(axis=1))
print(cm)

print("Accuracy = ",accuracy_score(Ytest.argmax(axis=1), test_predicted_label.argmax(axis=1)))
print("Accuracy = ",accuracy_score(Ytrain.argmax(axis=1), train_predicted_label.argmax(axis=1)))

In [ ]:
#to create domain feature vector
model = load_model('modeluc1/model_49.h5')
features=load(open('../features_uc_resnet152_updated.pkl','rb'))
predictions=dict()
for i in features:
    #print(i)
    x_n=np.array(features[i])
    #score = model.evaluate(x, y, batch_size=batch_size)
    preds=model.predict([x_n])
    predictions[i]=preds[0]

print(predictions['10_077'])
dump(predictions,open('domain_features.pkl','wb'))

In [ ]:
fea=load(open('domain_features.pkl','rb'))
print(fea)

In [ ]:
train_acc_x=np.zeros((15,1))
train_acc_y=np.zeros((15,1))
test_acc_x=np.zeros((15,1))
test_acc_y=np.zeros((15,1))
c=0
for j in range(15,20,1):
    print(j)
    model = load_model('/home/mh/mywork/classification/model_classify_categoryloss/model_' + str(j) + '.h5')
    predictions=list()
    count=0
    for i in train_vid_features:
        #print(i)
        x_n=np.array(train_vid_features[i])
        #score = model.evaluate(x, y, batch_size=batch_size)
        preds=model.predict([x_n])
        labels = np.argmax(preds, axis=-1)
        predictions.append(labels[0])
    predictions=np.array(predictions)
    pred_label = preprocessing.LabelBinarizer()
    predicted_label=pred_label.fit_transform(predictions)
    Ytrain=Ytrain.astype(int)
    train_acc_x[c]=c
    train_acc_y[c]=accuracy_score(Ytrain.argmax(axis=1), predicted_label.argmax(axis=1))
    predictions=list()
    count=0
    for i in test_vid_features:
        #print(i)
        x_n=np.array(test_vid_features[i])
        #score = model.evaluate(x, y, batch_size=batch_size)
        preds=model.predict([x_n])
        labels = np.argmax(preds, axis=-1)
        predictions.append(labels[0])
    predictions=np.array(predictions)
    pred_label = preprocessing.LabelBinarizer()
    predicted_label=pred_label.fit_transform(predictions)
    Ytest=Ytest.astype(int)
    test_acc_x[c]=c
    test_acc_y[c]=accuracy_score(Ytest.argmax(axis=1), predicted_label.argmax(axis=1))
    c+=1

In [ ]:
plt.figure()

plt.plot(train_acc_x, train_acc_y, 'g', label='Training accuracy')
plt.plot(test_acc_x, test_acc_y, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.show()

In [ ]:
print(train_acc_y)
print(test_acc_y)

In [ ]:
model = load_model('/home/mh/mywork/classification/model_classify_categoryloss/model_8.h5')
features=load(open('/home/mh/mywork/dataset/MSVD/features/class_features/msvd_caption_features.pkl','rb'))
#load training dataset (6K)
filename = '/home/mh/mywork/dataset/MSVD/all_video_ID.txt'
vid = load_set(filename)

In [ ]:
#accuracy for train:
predictions=list()
pr_vid=dict()
y_label=list()
count=0
for i in features:
    #print(i)
    x_n=np.array(features[i])

    #score = model.evaluate(x, y, batch_size=batch_size)
    preds=model.predict([x_n])
    labels = np.argmax(preds, axis=-1)
    predictions.append(labels[0])
    pr_vid[count]=i
    count+=1

predictions_train=np.array(predictions)
pred_label = preprocessing.LabelBinarizer()
predicted_label=pred_label.fit_transform(predictions)

print(len(pr_vid))
new_label=dict()
for i in range(len(vid)):
    new_label[pr_vid[i]]=predicted_label[i]

dump(new_label,open('msvd_class_weights_category.pkl','wb'))

In [ ]:
#get new class ids
from numpy import argmax
class_map={0:'actions',1:'animal',2:'cook',3:'music',4:'ride',5:'simpleactions'}
#train
train_domain=dict()
for c in class_map:
    train_domain[c]=list()
for i in Ytrain_label:
    class_label=argmax(Ytrain_label[i])
    train_domain[class_label].append(i)

#test
test_domain=dict()
for c in class_map:
    test_domain[c]=list()
for i in Ytest_label:
    class_label=argmax(Ytest_label[i])
    test_domain[class_label].append(i)


dump(train_domain,open('cluster_domain_train_ids.pkl','wb'))
dump(test_domain,open('cluster_domain_test_ids.pkl','wb'))

In [ ]:
print(train_domain[0])